<a href="https://colab.research.google.com/github/ysugiyama3/google_colab/blob/master/yale_dup_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Search Voyager records by ISBN**
This program automatically searches Voyager records by ISBN found in the first sheet of an Excel spreadsheet and returns bib id, title, mfhd id, mfhd location, call no, item id, iten enum, item location, and item status of matches found. The first column must be assigned for ISBN. The spreadsheet can have as many columns as necesary and must have column headers.

---
**How to run the program**
* Simply click the play button in the order.
* To start over, please go to the menu, go to "Runtime" and then select "Disconnect and delete runtime."
* To clear output, please go to "Edit" and then select "Clear all outputs."

Contact yukari.sugiyama@yale.edu if you have any issues or questions

---

In [ ]:
#@title <--- Upload Excel file { vertical-output: true, display-mode: "form" }

from pandas.core.common import index_labels_to_array
from google.colab import files
import pandas as pd
import requests
import json
from IPython.display import HTML, display
import time
import re

#===============================================================================
# defs
#===============================================================================

def progress(value, max=50000):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 40%'
        >
            {value}
        </progress>
        <br>{value}/{max}</br>
    """.format(value=value, max=max))

def clean_isbn(isbn):
    if isbn is None or pd.isnull(isbn):
        isbn = None
    elif len(str(isbn)) > 0:
        isbn = str(isbn)
        isbn = re.sub(r'[\(|\:|\.].*', '', isbn)
        isbn = re.sub(r'[^0-9Xx]', '', isbn) 
        isbn = isbn.zfill(10)
    else:
        isbn = None
    return isbn

def get_item_info(item):
    note = mfhdid = callno = itemid = enum = item_loc = status = ''
    mfhdid = item['mfhdid']
    callno = item['callno']
    itemid = item['itemid']  
    if itemid == 'NA':
        note = 'Held (but no item records)'
    else:
        enum = item['itemenum']
        if enum == 'NA':
            enum = ''
        item_loc = item['loccode']
        status = item['itemstatus']
        note = 'Held' 
    return note, mfhdid, callno, itemid, enum, item_loc, status

def check_bib_mfhd(bibid):
    mfhd_dict = dict()
    url = 'https://libapp.library.yale.edu/VoySearch/GetBibMfhdItem?bibid=' + str(bibid)
    r = requests.get(url)
    result = json.loads(r.text)
    record= result['records'][0]
    bibSuppressInOpac = record['bibSuppressInOpac']
    if bibSuppressInOpac == 'Y':
        return 'Suppressed'
    else:
        for holding in record['holding']:
            mfhdId = str(holding['mfhdId'])
            if mfhdId != 'NA':
                mfhdLocCode = holding['mfhdLocCode']
                mfhdSuppressInOpac = holding['mfhdSuppressInOpac']
            else:
                mfhdLocCode = 'NA'
                mfhdSuppressInOpac = 'NA' 
            if mfhdId not in mfhd_dict:
                mfhd_dict.update({mfhdId:(mfhdLocCode, mfhdSuppressInOpac)})
        return mfhd_dict

def check_records(records):
    global output_df
    for record in records:
        bibid = record['bibid']
        if bibid == 'NA':
            note = 'Check manually'
            output_df = output_df.append([input_df.iloc[index]],ignore_index=True)
            current_index_loc = output_df.index.size-1
            output_df.loc[current_index_loc, '[NOTE]'] = note
        else:
            bib_mfhd_check = check_bib_mfhd(bibid)
            try:
                title = record['title']   
                items = record['items']

                if bib_mfhd_check == 'Suppressed':
                    note = 'Bib record is suppressed'
                    output_df = output_df.append([input_df.iloc[index]],ignore_index=True)
                    current_index_loc = output_df.index.size-1
                    output_df.loc[current_index_loc, ['[NOTE]', '[BIB_ID]', '[TITLE]']] = [note, bibid, title]
                elif list(bib_mfhd_check.keys())[0] == 'NA':
                    note = 'Held (but no holdings records)'
                    output_df = output_df.append([input_df.iloc[index]],ignore_index=True)
                    current_index_loc = output_df.index.size-1
                    output_df.loc[current_index_loc, ['[NOTE]', '[BIB_ID]', '[TITLE]']] = [note, bibid, title]
                else:
                    for item in items:
                        note, mfhdid, callno, itemid, enum, item_loc, status = get_item_info(item)
                        if mfhdid in bib_mfhd_check:
                            mfhd_loc, mfhd_suppression = bib_mfhd_check.get(mfhdid)
                            if mfhd_suppression == 'Y':
                                note = 'Holdings record is suppressed'
                                output_df = output_df.append([input_df.iloc[index]],ignore_index=True)
                                current_index_loc = output_df.index.size-1
                                output_df.loc[current_index_loc, ['[NOTE]', '[BIB_ID]', '[TITLE]', '[MFHD_ID]', '[MFHD_LOC]', '[CALL_NO]', '[ITEM_ID]', '[ENUM]', '[ITEM_LOC]', '[STATUS]']] = [note, bibid, title, mfhdid, mfhd_loc, callno, itemid, enum, item_loc, status]
                            else:
                                output_df = output_df.append([input_df.iloc[index]],ignore_index=True)
                                current_index_loc = output_df.index.size-1
                                output_df.loc[current_index_loc, ['[NOTE]', '[BIB_ID]', '[TITLE]', '[MFHD_ID]', '[MFHD_LOC]', '[CALL_NO]', '[ITEM_ID]', '[ENUM]', '[ITEM_LOC]', '[STATUS]']] = [note, bibid, title, mfhdid, mfhd_loc, callno, itemid, enum, item_loc, status]
            except:
                note = 'Check manually'
                output_df = output_df.append([input_df.iloc[index]],ignore_index=True)
                current_index_loc = output_df.index.size-1
                output_df.loc[current_index_loc, '[NOTE]'] = note

def search_opac(isbn):
    global output_df
    if isbn is None or pd.isnull(isbn):
        note = 'No ISBN'
        output_df = output_df.append([input_df.iloc[index]],ignore_index=True)
        current_index_loc = output_df.index.size-1
        output_df.loc[current_index_loc, '[NOTE]'] = note
    else:
        url = 'http://libapp.library.yale.edu/VoySearch/GetBibItem?isxn=' + str(isbn)
        r = requests.get(url)
        result = json.loads(r.text)
        records= result['record']
        check_records(records)


#===============================================================================
# main
#===============================================================================

!pip install --upgrade xlrd &> /dev/null

# Upload an input Excel file
uploaded = files.upload()
input_name = str(list(uploaded.keys())[0])

# Read an input Excel file into a pandas DataFrame
input_df = pd.read_excel(input_name)

# Create an output Excel file based on input excel file
output_name = input_name.rsplit( ".", 1 )[0] + "_output.xlsx"

# Create an output DataFrame
output_df = pd.DataFrame(columns=input_df.columns)
output_df = pd.concat([output_df, pd.DataFrame(columns = ['[NOTE]', '[BIB_ID]', '[TITLE]', '[MFHD_ID]', '[MFHD_LOC]', '[CALL_NO]', '[ITEM_ID]', '[ENUM]', '[ITEM_LOC]', '[STATUS]'])], sort=False)

# count
total = input_df.index.size
count = 0

out = display(progress(0, total), display_id=True)

for index, row in input_df.iterrows():
    count += 1
    time.sleep(0.02)
    out.update(progress(count, total))
    isbn = clean_isbn(row[0])
    try:
        search_opac(isbn)
    except Exception as e:
        print(isbn, 'has error:', e)

try:
    output_df.to_excel(output_name, index=False)
except:
    output_name = output_name.rsplit( ".", 1 )[0] + '.csv'
    output_df.to_csv(output_name, index=False, encoding='utf-8')

print('\nDone! \U0001F60E\U0001F334\U0001F367\U0001f44D\n')
files.download(output_name)

In [ ]:
#@title <--- Preview the search results { vertical-output: true, display-mode: "form" }
output_df

,ISBN,TITLE,PUB_INFO,[NOTE],[BIB_ID],[TITLE],[MFHD_ID],[MFHD_LOC],[CALL_NO],[ITEM_ID],[ENUM],[ITEM_LOC],[STATUS]
0,9781588396174,Age of empires : art of the Qin and Han dynasties,"New York : Metropolitan Museum of Art, [2017]",Held,13110445,Age of empires : art of the Qin and Han dynast...,13185880,lsfmssr,Yns72 2017 +S85,11607502,,lsfmssr,Not Charged
1,9781588396174,Age of empires : art of the Qin and Han dynasties,"New York : Metropolitan Museum of Art, [2017]",Held,13110445,Age of empires : art of the Qin and Han dynast...,13186915,art,N7343.23 .S86 2017 (LC),11608427,,art,Renewed
2,9781788169745,Bacon in Moscow,"London : Cheerio, 2022.",Held,16184367,Bacon in Moscow /,16115077,ccl,ND497.B16 B57 2022 (LC),12777233,,ccl,Not Charged
3,9781788169745,Bacon in Moscow,"London : Cheerio, 2022.",Held (but no item records),16184367,Bacon in Moscow /,16153761,bacref,NJ18.B13 B57 2022 (LC),NA,,,
4,9781838510138,Beatrix Potter : drawn to nature,"London : V&A Publishing, 2021.",Held,16210851,Beatrix Potter : drawn to nature /,16140043,art,NC978.5.P6 A4 2021 (LC),12787171,,art,Charged
5,9780525510307,Between the world and me,"New York : Spiegel & Grau, 2017.",Check manually,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,9780735224933,Biased,"New York : Viking, an imprint of Penguin Rando...",Held,14072325,Biased : uncovering the hidden prejudice that ...,14079610,ccl,BF575.P9 E34 2019 (LC),12245373,,ccl,Not Charged
7,9780521186742 (pbk.),Chinese food,"Cambridge, UK ; New York [N.Y.] : Cambridge Un...",Held,10111443,Chinese food /,10392522,sml,GT2853.C6 L58 2011 (LC),9507818,,sml,"Not Charged,Missing"
8,9780593316375,Companion piece : a novel,"New York : Pantheon Books, [2022]",Held,16238819,Companion piece : a novel /,16165225,sml,PR6069.M4213 C66 2022 (LC),12803662,,sml,Discharged
9,9780593316375,Companion piece : a novel,"New York : Pantheon Books, [2022]",Held,16317362,Companion piece : a novel /,16238057,sml,ON ORDER,12825673,,sml,Not Charged
